## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Barrow,US,71.2906,-156.7887,-7.60,71,1,6.91,clear sky
1,1,Albany,US,42.6001,-73.9662,51.80,29,20,14.97,few clouds
2,2,Winslow,US,35.0242,-110.6974,70.00,19,1,28.77,clear sky
3,3,Lompoc,US,34.6391,-120.4579,57.20,51,1,19.57,clear sky
4,4,Mehamn,NO,71.0357,27.8492,31.93,84,100,14.18,light snow


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Meulaboh,ID,4.1363,96.1285,76.23,89,73,5.23,broken clouds
7,7,Puerto Ayora,EC,-0.7393,-90.3518,84.20,70,20,10.36,few clouds
11,11,Ranong,TH,9.9658,98.6348,76.05,88,78,2.37,broken clouds
15,15,Tandalti,SD,13.0167,31.8667,83.93,17,14,12.26,few clouds
19,19,Mahebourg,MU,-20.4081,57.7000,78.80,89,40,11.50,moderate rain
24,24,Rikitea,PF,-23.1203,-134.9692,80.06,72,5,6.22,clear sky
26,26,Marataizes,BR,-21.0433,-40.8244,76.05,87,0,9.13,clear sky
29,29,Sola,VU,-13.8833,167.5500,81.30,75,93,10.47,overcast clouds
31,31,Avarua,CK,-21.2078,-159.7750,84.20,79,20,9.22,light rain
33,33,Atuona,PF,-9.8000,-139.0333,80.33,79,15,13.38,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             196
City                   196
Country                196
Lat                    196
Lng                    196
Max Temp               196
Humidity               196
Cloudiness             196
Wind Speed             196
Current Description    196
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Meulaboh,ID,76.23,broken clouds,4.1363,96.1285,
7,Puerto Ayora,EC,84.20,few clouds,-0.7393,-90.3518,
11,Ranong,TH,76.05,broken clouds,9.9658,98.6348,
15,Tandalti,SD,83.93,few clouds,13.0167,31.8667,
19,Mahebourg,MU,78.80,moderate rain,-20.4081,57.7000,
24,Rikitea,PF,80.06,clear sky,-23.1203,-134.9692,
26,Marataizes,BR,76.05,clear sky,-21.0433,-40.8244,
29,Sola,VU,81.30,overcast clouds,-13.8833,167.5500,
31,Avarua,CK,84.20,light rain,-21.2078,-159.7750,
33,Atuona,PF,80.33,few clouds,-9.8000,-139.0333,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [18]:
# 8a. Create the output File (CSV)
output_data_file = "../vacation_search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))